In [3]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
import pickle
import joblib
import numpy as np
import pandas as pd
import librosa
from scipy import signal

# Setting paths
BASE_PATH = '/content/drive/MyDrive/cough_detection'
MODEL_PATH = f'{BASE_PATH}/baseline_model.pkl'
DATASET_PATH = f'{BASE_PATH}/dataset/Multimodal Cough Dataset/'

print("✅ Paths set")

✅ Paths set


In [15]:
# Loading baseline
with open(MODEL_PATH, 'rb') as f:
    model = pickle.load(f)

print(f"✅ Baseline model loaded: {type(model)}")

✅ Baseline model loaded: <class 'sklearn.linear_model._logistic.LogisticRegression'>


In [16]:
# FUNCTIONS TO CALCULATE VTI SCORE
def get_audio_path(participant, trial_name, trial_folder):
    """Get correct audio path based on trial type"""

    if trial_name == "No_Talking":
        in_path = f"{DATASET_PATH}{participant}/{trial_folder}/{participant}_No_Talking_In.wav"
        if os.path.exists(in_path):
            return in_path, "in"
        out_path = f"{DATASET_PATH}{participant}/{trial_folder}/{participant}_No_Talking_Out.wav"
        if os.path.exists(out_path):
            return out_path, "out"

    elif trial_name == "Talking":
        out_path = f"{DATASET_PATH}{participant}/{trial_folder}/{participant}_Talking_Out.wav"
        if os.path.exists(out_path):
            return out_path, "out"

    return None, None

# IMU extraction function
def extract_imu_features_from_file(participant, trial_folder):
    """Extract comprehensive IMU features"""
    imu_features = {}

    # Accelerometer
    accel_path = f"{DATASET_PATH}{participant}/{trial_folder}/Accelerometer.csv"
    if os.path.exists(accel_path):
        accel = pd.read_csv(accel_path)
        x_col = [col for col in accel.columns if 'x-axis' in col][0]
        y_col = [col for col in accel.columns if 'y-axis' in col][0]
        z_col = [col for col in accel.columns if 'z-axis' in col][0]

        mag = np.sqrt(accel[x_col]**2 + accel[y_col]**2 + accel[z_col]**2)

        imu_features.update({
            'accel_mean': float(mag.mean()),
            'accel_std': float(mag.std()),
            'accel_max': float(mag.max()),
            'accel_energy': float(np.sum(mag**2)),
        })

        # Respiratory rate
        b, a = signal.butter(3, 0.5/(50), btype='low')
        breathing = signal.filtfilt(b, a, mag)
        peaks, _ = signal.find_peaks(breathing, distance=50)
        imu_features['respiratory_rate'] = len(peaks) / (len(accel)/50) * 60 if len(peaks) > 0 else 0
    else:
        imu_features.update({'accel_mean': 0, 'accel_std': 0, 'accel_max': 0,
                           'accel_energy': 0, 'respiratory_rate': 0})

    # Gyroscope
    gyro_path = f"{DATASET_PATH}{participant}/{trial_folder}/Gyroscope.csv"
    if os.path.exists(gyro_path):
        gyro = pd.read_csv(gyro_path)
        x_col = [col for col in gyro.columns if 'x-axis' in col][0]
        y_col = [col for col in gyro.columns if 'y-axis' in col][0]
        z_col = [col for col in gyro.columns if 'z-axis' in col][0]

        rot_mag = np.sqrt(gyro[x_col]**2 + gyro[y_col]**2 + gyro[z_col]**2)

        imu_features.update({
            'gyro_mean': float(rot_mag.mean()),
            'gyro_max': float(rot_mag.max()),
            'cough_force_proxy': float(rot_mag.max() * imu_features.get('accel_max', 1)),
        })
    else:
        imu_features.update({'gyro_mean': 0, 'gyro_max': 0, 'cough_force_proxy': 0})

    # Magnetometer
    mag_path = f"{DATASET_PATH}{participant}/{trial_folder}/Magnetometer.csv"
    if os.path.exists(mag_path):
        mag_field = pd.read_csv(mag_path)
        x_col = [col for col in mag_field.columns if 'x-axis' in col][0]
        y_col = [col for col in mag_field.columns if 'y-axis' in col][0]
        z_col = [col for col in mag_field.columns if 'z-axis' in col][0]

        imu_features.update({
            'orientation_stability': float(mag_field[x_col].std() +
                                           mag_field[y_col].std() +
                                           mag_field[z_col].std()),
        })
    else:
        imu_features['orientation_stability'] = 0

    return imu_features

# Cough detection function
def detect_coughs_with_force(audio_path, model, sr=750):
    """Detect coughs in audio file"""
    try:
        audio, sr = librosa.load(audio_path, sr=sr)
    except:
        return [], []

    window_size = int(2.0 * sr)
    step_size = int(0.5 * sr)

    cough_times = []
    cough_confidences = []

    for start in range(0, len(audio) - window_size, step_size):
        segment = audio[start:start+window_size]

        mel_spec = librosa.feature.melspectrogram(
            y=segment.astype(float), sr=sr, n_mels=64,
            n_fft=256, hop_length=64
        )
        log_mel = librosa.power_to_db(mel_spec)
        features = log_mel.flatten().reshape(1, -1).astype(np.float32)

        prob = model.predict_proba(features)[0, 1]

        if prob > 0.5:
            energy = segment**2
            peak_idx = np.argmax(energy)
            exact_time = (start + peak_idx) / sr
            cough_times.append(exact_time)
            cough_confidences.append(float(prob))

    return cough_times, cough_confidences

# VTI calculation
def calculate_enhanced_vti(participant_data):
    suppressibility_score = 100 * (1 - min(participant_data['suppression_ratio'], 1))

    imu_score = 0
    if participant_data['cough_force'] > 0:
        force_score = min(100, participant_data['cough_force'] * 0.01)  # Adjusted scaling
        imu_score += 0.3 * force_score

    if participant_data['respiratory_rate'] > 0:
        resp_score = max(0, 100 - (participant_data['respiratory_rate'] * 2))
        imu_score += 0.3 * resp_score

    if participant_data['movement_consistency'] > 0:
        consistency_score = min(100, 100 - participant_data['movement_consistency'] * 10)
        imu_score += 0.2 * consistency_score

    orient_score = 100 - min(100, participant_data['orientation_stability'] * 1000)
    imu_score += 0.2 * max(0, orient_score)

    enhanced_vti = 0.5 * suppressibility_score + 0.5 * imu_score

    return {
        'vti_audio': suppressibility_score,
        'vti_imu': imu_score,
        'vti_enhanced': enhanced_vti
    }

print("✅ All functions loaded")

✅ All functions loaded


In [17]:
#test on first participant
# Test on first participant
participant = "005"
print(f"\n{'='*50}")
print(f"📊 Testing Participant {participant}")
print(f"{'='*50}")

trial_data = {}

for trial_name, trial_folder in [('No_Talking', 'Trial_1_No_Talking'),
                                  ('Talking', 'Trial_2_Talking')]:

    audio_path, mic_type = get_audio_path(participant, trial_name, trial_folder)

    if audio_path:
        print(f"\n   📁 {trial_name}: using {mic_type} microphone")

        cough_times, confidences = detect_coughs_with_force(audio_path, model)
        imu_feat = extract_imu_features_from_file(participant, trial_folder)

        audio, sr = librosa.load(audio_path, sr=750)
        duration = len(audio) / sr

        trial_data[trial_name] = {
            'cough_count': len(cough_times),
            'cough_rate': len(cough_times) / duration * 60,
            **imu_feat
        }

        print(f"      Coughs: {len(cough_times)} ({trial_data[trial_name]['cough_rate']:.1f}/min)")
        print(f"      Cough force: {imu_feat.get('cough_force_proxy', 0):.2f}")

if 'No_Talking' in trial_data and 'Talking' in trial_data:
    no_rate = trial_data['No_Talking']['cough_rate']
    talk_rate = trial_data['Talking']['cough_rate']
    ratio = talk_rate / no_rate if no_rate > 0 else 1

    participant_data = {
        'suppression_ratio': ratio,
        'cough_force': trial_data['No_Talking'].get('cough_force_proxy', 0),
        'respiratory_rate': trial_data['No_Talking'].get('respiratory_rate', 0),
        'movement_consistency': trial_data['No_Talking'].get('accel_std', 0),
        'orientation_stability': trial_data['No_Talking'].get('orientation_stability', 0)
    }

    vti = calculate_enhanced_vti(participant_data)
    print(f"\n   🧠 VTI: {vti['vti_enhanced']:.1f}/100")


📊 Testing Participant 005

   📁 No_Talking: using in microphone
      Coughs: 0 (0.0/min)
      Cough force: 7475.04

   📁 Talking: using out microphone
      Coughs: 0 (0.0/min)
      Cough force: 3778.33

   🧠 VTI: 41.9/100


In [18]:
participants = ['005', '006', '007', '008', '009', '010',
                '011', '012', '013', '014', '015', '016', '017']

all_results = []

for participant in participants:
    print(f"\n{'='*50}")
    print(f"📊 Processing {participant}")

    trial_data = {}

    for trial_name, trial_folder in [('No_Talking', 'Trial_1_No_Talking'),
                                      ('Talking', 'Trial_2_Talking')]:

        audio_path, mic_type = get_audio_path(participant, trial_name, trial_folder)
        if not audio_path:
            continue

        cough_times, _ = detect_coughs_with_force(audio_path, model)
        imu_feat = extract_imu_features_from_file(participant, trial_folder)

        audio, sr = librosa.load(audio_path, sr=750)
        duration = len(audio) / sr

        trial_data[trial_name] = {
            'cough_rate': len(cough_times) / duration * 60,
            **imu_feat
        }

    if 'No_Talking' in trial_data and 'Talking' in trial_data:
        no_rate = trial_data['No_Talking']['cough_rate']
        talk_rate = trial_data['Talking']['cough_rate']
        ratio = talk_rate / no_rate if no_rate > 0 else 1

        data = {
            'suppression_ratio': ratio,
            'cough_force': trial_data['No_Talking'].get('cough_force_proxy', 0),
            'respiratory_rate': trial_data['No_Talking'].get('respiratory_rate', 0),
            'movement_consistency': trial_data['No_Talking'].get('accel_std', 0),
            'orientation_stability': trial_data['No_Talking'].get('orientation_stability', 0)
        }

        vti = calculate_enhanced_vti(data)

        all_results.append({
            'participant': participant,
            'vti': round(vti['vti_enhanced'], 1),
            'ratio': round(ratio, 2),
            'cough_force': round(data['cough_force'], 2),
            'resp_rate': round(data['respiratory_rate'], 1)
        })

# Save results
df = pd.DataFrame(all_results)
df.to_csv(f'{BASE_PATH}/vti_baseline_results.csv', index=False)
print("\n✅ Results saved!")
print(df)


📊 Processing 005

📊 Processing 006

📊 Processing 007

📊 Processing 008

📊 Processing 009

📊 Processing 010

📊 Processing 011

📊 Processing 012

📊 Processing 013

📊 Processing 014

📊 Processing 015

📊 Processing 016

📊 Processing 017

✅ Results saved!
   participant   vti  ratio  cough_force  resp_rate
0          005  41.9      1      7475.04       13.3
1          006  42.6      1      7994.09       13.2
2          007  34.1      1      2329.43       13.5
3          008  32.1      1      1196.11       14.5
4          009  34.4      1      2538.32       13.3
5          010  33.1      1      2077.88       15.6
6          011  38.2      1      4942.92       13.2
7          012  35.5      1      3412.64       13.9
8          013  32.9      1      1981.73       15.6
9          014  31.3      1       726.34       15.4
10         015  36.7      1      4394.93       15.0
11         016  33.4      1      1924.63       13.6
12         017  10.0      1         0.00        0.0
